# Morningstar: Entraînement sur Google Colab

Ce notebook permet d'entraîner le modèle Morningstar en utilisant les ressources GPU de Google Colab.

## Options disponibles
1. **Mode échange crypto** (KuCoin/Bybit recommandés):
   - Télécharge les données OHLCV directement depuis l'échange
   - Nécessite un compte sur l'échange sélectionné
   - Risque de restrictions géographiques (erreur 451)

2. **Mode dataset préparé**:
   - Utilise un fichier CSV préalablement préparé
   - Plus stable et rapide
   - Requiert un lien valide vers le dataset

## Guide d'utilisation
1. **Configuration**:
   - Choisir le mode dans la première cellule
   - Pour le mode dataset, fournir l'URL du fichier CSV
   
2. **Installation**:
   - Exécuter toutes les cellules dans l'ordre
   
3. **Dépannage**:
   - Erreur 451: Changer d'échange (essayer Bybit si KuCoin échoue)
   - Données manquantes: Vérifier les dates et la paire de trading
   
> **Note**: Pour obtenir le dataset préparé, contactez l'administrateur du projet.

## 1. Configuration

In [ ]:
# Vérification de l'environnement
try:
    import google.colab
    IN_COLAB = True
    print("Environnement Colab détecté - Toutes les fonctionnalités sont disponibles")
except:
    IN_COLAB = False
    print("Mode local - Certaines fonctionnalités Colab ne sont pas disponibles")

# --- Options ---
use_prepared_data = True  # False pour télécharger depuis un échange

if not use_prepared_data:
    # Paramètres pour téléchargement depuis échange
    exchange_name = 'kucoin'  # Recommandé: 'kucoin' ou 'bybit'
    pair = 'BTC/USDT'
    timeframe = '1h'
    start_date = '2022-01-01'
    end_date = '2024-01-01'
else:
    # Charger le dataset préparé
    !wget https://votre-lien-vers-dataset/dataset_prepared.csv
    dataset_path = 'dataset_prepared.csv'

## 2. Installation des Dépendances

In [ ]:
!pip install -q tensorflow pandas numpy
!git clone https://github.com/Cabrel10/eva001.git
%cd eva001
!pip install -q .

## 3. Préparation des Données

In [ ]:
from Morningstar.model.architecture.morningstar_model import MorningstarTradingModel
from Morningstar.configs.morningstar_config import MorningstarConfig
import pandas as pd
import numpy as np

if use_prepared_data:
    # Chargement du dataset préparé
    data = pd.read_csv(dataset_path)
    # Vérification des colonnes requises
    required_cols = ['open', 'high', 'low', 'close', 'volume', 'rsi', 'macd']
    assert all(col in data.columns for col in required_cols), "Colonnes manquantes dans le dataset"
    print(f"Dataset chargé avec succès. Shape: {data.shape}")
else:
    # Téléchargement depuis l'échange
    from Morningstar.utils.data_manager import ExchangeDataManager
    manager = ExchangeDataManager(exchange_name)
    data = manager.load_data(pair, timeframe, start_date, end_date)
    print(f"Données téléchargées depuis {exchange_name}. Shape: {data.shape}")

# Vérification finale des données
print("\nAperçu des données:")
print(data.head())
print("\nStatistiques descriptives:")
print(data.describe())

## 4. Entraînement du Modèle

In [ ]:
# Sauvegarde et téléchargement du modèle
model_path = 'morningstar_trained.h5'
model_instance.model.save(model_path)
print(f"\nModèle entraîné sauvegardé sous {model_path}")

if IN_COLAB:
    from google.colab import files
    print("Téléchargement du modèle en cours...")
    files.download(model_path)
    print("Modèle téléchargé avec succès depuis Colab")
else:
    print(f"Modèle disponible localement: {model_path}")

# Vérification finale
try:
    from tensorflow.keras.models import load_model
    loaded_model = load_model(model_path)
    print("\nVérification réussie - Modèle fonctionnel et portable")
    print("Le modèle est prêt à être utilisé en production")
except Exception as e:
    print(f"\nErreur lors de la vérification: {str(e)}")
    print("Veuillez vérifier l'intégrité du modèle sauvegardé")